In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory("Brain_tumor_images/",
                                                              validation_split=0.2,
                                                              subset='training',
                                                              seed = 42,
                                                              image_size=(150,150),
                                                              batch_size=32)
val_ds = tf.keras.preprocessing.image_dataset_from_directory("Brain_tumor_images/",
                                                            validation_split=0.2,
                                                            subset='validation',
                                                            seed = 42,
                                                            image_size=(150,150),
                                                            batch_size=32)
class_names = train_ds.class_names
print(class_names)

scaling = keras.layers.Rescaling(1/255.)
train_ds = train_ds.map(lambda x,y:(scaling(x),y))
val_ds = val_ds.map(lambda x,y:(scaling(x),y))

Found 804 files belonging to 2 classes.
Using 644 files for training.
Found 804 files belonging to 2 classes.
Using 160 files for validation.
['Normal', 'Tumor']


In [3]:
data_augmentation = keras.Sequential(
    [keras.layers.RandomFlip("horizontal"), keras.layers.RandomRotation(0.1),]
)

In [4]:
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
) 
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)  # Apply random data augmentation
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 150, 150, 3)       0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                             

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"])
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
base_model.trainable = True
model.summary()

model.compile(
    optimizer="adam",  # Low learning rate
    loss="binary_crossentropy",
    metrics=['accuracy'],
)

epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=val_ds)